In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import classification_report

In [2]:
df=pd.read_csv('labels.csv', index_col=0)

In [3]:
df.head()

,image_name,text_ocr,text_corrected,overall_sentiment
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,very_positive
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,very_positive
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,positive
3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,positive
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,neutral


In [4]:
# Import necessary libraries
import os
import skimage.io as io
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES=True

# Set the path to the images folder in your Google Drive
folder_path = 'images\\'

# List the files in the folder
files = os.listdir(folder_path)
# Load the images
images = []
for file in files:
     #Load the image using OpenCV
     img = io.imread(folder_path+file)
     #Add the image to the list
     images.append(img)
 # Read the images in the folder with .png and .jpg extensions

c:\Users\huzai\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\TiffImagePlugin.py:845: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


In [5]:
# Load the dataset
X_images = images  # The images data
X_text = df["text_corrected"]  # The text data
y = df["overall_sentiment"]  # The labels

In [6]:
import numpy as np
from PIL import Image
image_array=[]
for i in range(len(images)):
    image=Image.fromarray(images[i]).convert("L")
    image=image.resize((100, 100))
    image=np.array(image)
    image_array.append(image)

In [7]:
print(len(image_array))

6992


In [8]:
print(len(y))

6992


In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(image_array, y, test_size=0.25, random_state=123)

In [10]:
x_train = np.array(x_train)
x_test = np.array(x_test)

samples, height, width = x_train.shape
x_train = x_train.reshape(samples, height * width)

samples, height, width = x_test.shape
x_test = x_test.reshape(samples, height * width)

In [11]:
rf=RandomForestClassifier(n_estimators=100, random_state=123)
rf.fit(x_train, y_train)

RandomForestClassifier(random_state=123)

In [12]:
rf_pred=rf.predict(x_test)
print("Random Forest Accuracy:", accuracy_score(y_test, rf_pred))
print(classification_report(y_test, rf_pred, zero_division=1))

Random Forest Accuracy: 0.42105263157894735
               precision    recall  f1-score   support

     negative       0.00      0.00      0.00       124
      neutral       0.32      0.18      0.23       543
     positive       0.45      0.82      0.58       779
very_negative       1.00      0.00      0.00        42
very_positive       0.00      0.00      0.00       260

     accuracy                           0.42      1748
    macro avg       0.35      0.20      0.16      1748
 weighted avg       0.32      0.42      0.33      1748



In [13]:
confusion_matrix(y_test, rf_pred)

array([[  0,  25,  99,   0,   0],
       [  1,  98, 441,   0,   3],
       [  0, 133, 638,   0,   8],
       [  0,   6,  36,   0,   0],
       [  0,  45, 215,   0,   0]], dtype=int64)

In [14]:
#Model for Image
#Logistic Regression
log_reg = LogisticRegression(max_iter=1000, random_state=123)
log_reg.fit(x_train, y_train)

c:\Users\huzai\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000, random_state=123)

In [15]:
from sklearn.metrics import classification_report
log_pred = log_reg.predict(x_test)
print("Logistic Regression Accuracy: ", accuracy_score(y_test, log_pred))
print(classification_report(y_test, log_pred))

Logistic Regression Accuracy:  0.33581235697940504
               precision    recall  f1-score   support

     negative       0.04      0.04      0.04       124
      neutral       0.32      0.32      0.32       543
     positive       0.45      0.48      0.47       779
very_negative       0.04      0.02      0.03        42
very_positive       0.13      0.12      0.13       260

     accuracy                           0.34      1748
    macro avg       0.20      0.20      0.20      1748
 weighted avg       0.33      0.34      0.33      1748



In [16]:
confusion_matrix(y_test, log_pred)

array([[  5,  38,  61,   3,  17],
       [ 32, 172, 251,   9,  79],
       [ 55, 236, 377,   7, 104],
       [  3,  11,  20,   1,   7],
       [ 24,  77, 121,   6,  32]], dtype=int64)

In [17]:
from sklearn.neighbors import KNeighborsClassifier
#K Nearest Neighbors
knn = KNeighborsClassifier(n_neighbors=80, weights='distance', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None)
knn.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=80, weights='distance')

In [18]:
knn_pred = knn.predict(x_test)
print("K Nearest Neighbors Accuracy: ", accuracy_score(y_test, knn_pred))
print(classification_report(y_test, knn_pred, zero_division=1))

K Nearest Neighbors Accuracy:  0.42620137299771166
               precision    recall  f1-score   support

     negative       0.00      0.00      0.00       124
      neutral       0.31      0.14      0.19       543
     positive       0.45      0.86      0.59       779
very_negative       1.00      0.00      0.00        42
very_positive       0.00      0.00      0.00       260

     accuracy                           0.43      1748
    macro avg       0.35      0.20      0.16      1748
 weighted avg       0.32      0.43      0.32      1748



In [19]:
confusion_matrix(y_test, knn_pred)

array([[  0,  16, 108,   0,   0],
       [  1,  74, 467,   0,   1],
       [  0, 105, 671,   0,   3],
       [  0,   3,  39,   0,   0],
       [  0,  41, 219,   0,   0]], dtype=int64)

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
x=df["text_corrected"]
y=df["overall_sentiment"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2 , random_state= 40)

#converting the text to vectors

vectorizer = CountVectorizer()
x_train = vectorizer.fit_transform(x_train.values.astype('U'))
x_test = vectorizer.transform(x_test.values.astype('U'))

In [21]:
from sklearn.naive_bayes import GaussianNB
#Model for Text
#Gaussian Naive Bayes
gnb = GaussianNB()

gnb.fit(x_train.toarray(), y_train)

GaussianNB()

In [22]:
from sklearn.metrics import classification_report
gnb_pred = gnb.predict(x_test.toarray())
print("Gaussian Naive Bayes Accuracy: ", accuracy_score(y_test, gnb_pred))
print(classification_report(y_test, gnb_pred))

Gaussian Naive Bayes Accuracy:  0.24017155110793423
               precision    recall  f1-score   support

     negative       0.03      0.07      0.04        92
      neutral       0.27      0.28      0.28       433
     positive       0.43      0.26      0.32       637
very_negative       0.03      0.06      0.04        35
very_positive       0.15      0.20      0.17       202

     accuracy                           0.24      1399
    macro avg       0.18      0.17      0.17      1399
 weighted avg       0.31      0.24      0.26      1399



In [23]:
from sklearn.tree import DecisionTreeClassifier
#Decision Tree
dt = DecisionTreeClassifier()
dt.fit(x_train.toarray(), y_train)

DecisionTreeClassifier()

In [24]:
dt_pred = dt.predict(x_test.toarray())
print("Decision Tree Accuracy: ", accuracy_score(y_test, dt_pred))
print(classification_report(y_test, dt_pred))

Decision Tree Accuracy:  0.363831308077198
               precision    recall  f1-score   support

     negative       0.05      0.03      0.04        92
      neutral       0.31      0.30      0.31       433
     positive       0.45      0.54      0.49       637
very_negative       0.00      0.00      0.00        35
very_positive       0.19      0.14      0.16       202

     accuracy                           0.36      1399
    macro avg       0.20      0.20      0.20      1399
 weighted avg       0.33      0.36      0.35      1399



In [25]:
from sklearn.linear_model import SGDClassifier
sgd=SGDClassifier(loss="hinge", penalty="l2", max_iter=10000, random_state=50, n_jobs=-1)
sgd.fit(x_train.toarray(), y_train)

SGDClassifier(max_iter=10000, n_jobs=-1, random_state=50)

In [26]:
sgd_pred = sgd.predict(x_test.toarray())
print("SGDClassifier Accuracy: ", accuracy_score(y_test, sgd_pred))
print(classification_report(y_test, sgd_pred))

SGDClassifier Accuracy:  0.3438170121515368
               precision    recall  f1-score   support

     negative       0.04      0.03      0.04        92
      neutral       0.32      0.29      0.31       433
     positive       0.45      0.51      0.47       637
very_negative       0.00      0.00      0.00        35
very_positive       0.14      0.15      0.15       202

     accuracy                           0.34      1399
    macro avg       0.19      0.20      0.19      1399
 weighted avg       0.33      0.34      0.33      1399



In [27]:
#Confusion Matrix
cm = confusion_matrix(y_test, sgd_pred)
print('Confusion Matrix: ')
print(cm)

Confusion Matrix: 
[[  3  25  53   1  10]
 [ 16 125 214   2  76]
 [ 39 176 322   2  98]
 [  2  11  20   0   2]
 [ 11  49 111   0  31]]


In [29]:
from sklearn.ensemble import VotingClassifier
#Voting Classifier
voting_clf = VotingClassifier(estimators=[('lr', log_reg),('knn', knn),('gnb', gnb),('dt', dt), ('rf', rf), ('sgd', sgd)], voting='hard')
voting_clf.fit(x_train.toarray(), y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(max_iter=1000,
                                                 random_state=123)),
                             ('knn',
                              KNeighborsClassifier(n_neighbors=80,
                                                   weights='distance')),
                             ('gnb', GaussianNB()),
                             ('dt', DecisionTreeClassifier()),
                             ('rf', RandomForestClassifier(random_state=123)),
                             ('sgd',
                              SGDClassifier(max_iter=10000, n_jobs=-1,
                                            random_state=50))])

In [32]:
voting_clf_pred = voting_clf.predict(x_test.toarray())
print("Majority Vote Accuracy: ", accuracy_score(y_test, voting_clf_pred))
print(classification_report(y_test, voting_clf_pred, zero_division=1))

Majority Vote Accuracy:  0.4045746962115797
               precision    recall  f1-score   support

     negative       0.00      0.00      0.00        92
      neutral       0.32      0.25      0.28       433
     positive       0.45      0.71      0.55       637
very_negative       1.00      0.00      0.00        35
very_positive       0.16      0.03      0.05       202

     accuracy                           0.40      1399
    macro avg       0.39      0.20      0.18      1399
 weighted avg       0.35      0.40      0.35      1399



In [ ]:
import pickle
pickle.dump(dt, open("dt.pkl", "wb"))
pickle.dump(sgd, open("sgd.pkl", "wb"))
pickle.dump(gnb, open("gnb.pkl", "wb"))
pickle.dump(knn, open("knn.pkl", "wb"))
pickle.dump(rf, open("rf.pkl", "wb"))
pickle.dump(log_reg, open("log_reg.pkl", "wb"))
pickle.dump(vectorizer, open("vect.pkl", "wb"))